# Plugin 与 function
插件是Semantic Kernel最重要的基本构建模块之一。通过插件，您可以将功能封装到一个单一的功能单元中，然后由内核Kernel运行。

插件可以包含本地代码和通过语义函数向AI服务发送的请求。

在一个插件内，您可以创建两种类型的函数：Semantic Function 和 Native Function

## Semantic Function
简单的说，Semantic Function实质是封装的 Prompt Template 与 API 参数.

在上个Lab中，我们见到的就是 Semantic Function。

当时为简单起见的我们将 Semantic Function 定义在代码当中， 并省略了参数部分。

在此Lab, 我们将更详细的介绍 Semantic Function， 它的不同形态与参数含义。



1: 首先还是创建Kernel，这部分与上个Lab一样、

In [1]:
// 安装 nuget 包
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: Microsoft.Extensions.Logging.Console, 6.0.0.0"

// 如遇到这个错误请忽略并继续。 Error: Microsoft.Extensions.Logging.Console version 6.0.0.0 cannot be added because version 6.0.0 was added previously

Installed Packages Microsoft.Extensions.Logging.Console, 6.0.0 Microsoft.SemanticKernel, 1.0.0-beta8

In [2]:
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;

//Create Kernel builder
var builder = new KernelBuilder();

//此lab中我们使用OPENAI服务. 如使用Azure OPENAI，请取消以下注释
/*
var model = "gpt-35-turbo";
var apiKey = "<your apikey>";
azureEndpoint="<your azure openai endpoint>";
ArgumentNullException.ThrowIfNull(apiKey);
ArgumentNullException.ThrowIfNull(azureEndpoint);
builder.WithAzureOpenAIChatCompletionService(model, azureEndpoint, apiKey);
*/
var model = "gpt-3.5-turbo-1106";
var apiKey = Environment.GetEnvironmentVariable("OPENAI__APIKEY");

ArgumentNullException.ThrowIfNull(apiKey);

var loggerFactory = LoggerFactory.Create(builder => { 
    builder.SetMinimumLevel(LogLevel.Information);  
    builder.AddConsole(); });

var kernel = builder
.WithLoggerFactory(loggerFactory)
.WithOpenAIChatCompletionService(model, apiKey)
.Build();

2： 此处的 JokeFunc 是以文本形式创建在 ./plugins/FunPlugin目录下。

我们来看看 ./plugins/FunPlugin目录
FunPlugin 是 Plugin 的名称，它下面的子目录为Semantic Function. 
每个Semantic Function 都包含两个文件 skprompt.txt, config.json


skprompt.txt 的内容即是 Prompt Template. 


```
"请生成一个关于 {{ $input}} 的笑话, 要求用{{ $style }}幽默";
```

 $input， $style  将由输入的variables 替换。

config.json 很好理解。
需要注意的是 "temperature". 这个阐述范围从0 到 1 表示模型可以自由发挥的程度。如需要高一致性的结果，通常会将temperature 设成 0.

```

{
    "schema": 1,
    "description": "Generate a funny joke",
    "models": [
      {
        "max_tokens": 1000,
        "temperature": 0.9,
        "top_p": 0.0,
        "presence_penalty": 0.0,
        "frequency_penalty": 0.0
      }
    ],
    "input": {
      "parameters": [
        {
          "name": "input",
          "description": "笑话的主题",
          "defaultValue": ""
        }
        ，
        {
          "name": "style",
          "description": "幽默类型"
        }
      ]
    }
  }
  
```

3： 将Plugin导入Kernel

In [3]:
using System.IO;
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(),  "plugins");
var funPluginFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "FunPlugin");
var jokeFunc = funPluginFunctions["joke"];


注意，上个lab的 JokeFunc 我们只用到一个`$input`变量。所有输入值如无变量名则默认是`$input`. 此处我们需要输入两个变量, 需要用到 ContextVariables 来传递。

In [6]:
using Microsoft.SemanticKernel.Orchestration;
var humberContext = new ContextVariables
{
    ["input"] = "加拿大",
    ["style"] = "黑色"
};

var result = await kernel.RunAsync(jokeFunc, humberContext);


Console.WriteLine(result.GetValue<string>());
result.FunctionResults.First()

为了避免冷战升级，加拿大政府决定将所有冰球队解散。结果，全国范围内发生了一场“冰球战争”，只不过所有人都在道歉。


为了避免冷战升级，加拿大政府决定将所有冰球队解散。结果，全国范围内发生了一场“冰球战争”，只不过所有人都在道歉。 FunctionName joke PluginName FunPlugin Metadata key type value ModelResults Microsoft.SemanticKernel.Orchestration.ModelResult[] index value 0 Microsoft.SemanticKernel.Orchestration.ModelResult RenderedPrompt System.String 请生成一个关于 加拿大 的笑话, 要求用黑色幽默

3: 我们也可以在一个Semantic Function中来调用另外的 Function.请看以下例子。  `{{FunPlugin.joke $input}}`

In [11]:
const string translateFuncDefinition = "请讲以下输入翻译成英文： {{FunPlugin.joke $input}}";

var translateToEngFunc = kernel.CreateSemanticFunction(translateFuncDefinition);
var result2 = await kernel.RunAsync("United States", jokeFunc, translateToEngFunc);
Console.WriteLine(result2.GetValue<string>());

Why are Americans always late? Because they are always looking for their own free time! So, if you make an appointment to meet an American, it's best to push the time back an hour so they have enough time to find their own freedom!


# Native Function

In [15]:
using System.ComponentModel;
using System.Globalization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
public class Text
{
    private IDictionary<string, string> mem = new Dictionary<string, string>();

    [SKFunction, Description("把文本转换存入字典")]
    public async void MemorizeToDict(string subject, SKContext context)
    {
        var key = subject.Trim().ToLowerInvariant();
        var jokeFunc = context.Plugins["FuncPlugin"]["joke"];
        var joke = await context.Runner.RunAsync("OpenAI", jokeFunc);
        mem[key] = joke;
    }


}

Error: (13,32): error CS1061: 'SKContext' does not contain a definition for 'Plugins' and no accessible extension method 'Plugins' accepting a first argument of type 'SKContext' could be found (are you missing a using directive or an assembly reference?)

## 实践：创建翻译法文的Semantic Function 并储存在 ./plugins/WritingPlug/TranslateFr下